In [21]:
import langchain
print(langchain.__version__)

0.2.5


In [22]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from config import OPENAI_API_KEY

In [23]:
model = ChatOpenAI(temperature=0.3)

In [16]:
# Define your desired data structure.
# class Joke(BaseModel):
#     setup: str = Field(description="question to set up a joke")
#     punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n return a JSON dictionary with a key setup that contains the question to set up a joke and a punchline that answer to resolve the joke",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired!'}

In [26]:
class Section(BaseModel):
    section: str = Field(description="text section relevant for identification as sub_category")
    category: str = Field(description="category that best describes the section")
    sub_category: str = Field(description="specifies a subtype or aspect within the category")
    explanation: str = Field(description="brief explanation why this section is identified")
    
analysis_template = """
    Identify all text sections relevant to these categories: {categories}.
    Create a JSON object containing a list of identified 'sections' with the key 'sections'.
    Each 'section' is in the format: {format_instructions}
    If none of the given categories are clearly identifiable in the text, return an empty object.
    Text: {text}
"""

categories = "bias, logical fallacy"
text = """
Information that I saw online about COVID-19 disease causing sterility in the summer was removed.
I heard it’s because of lawsuits related to Bill Gates and the vaccines in Africa.
Does this have to do with the COVID-19 vaccines causing infertility?
"""

parser = JsonOutputParser(pydantic_object=Section)

prompt = PromptTemplate(
    template = analysis_template,
    partial_variables={"format_instructions": parser.get_format_instructions()},
    input_variables=['categories', 'text'],
)

chain = prompt | model | parser

chain.invoke({"categories": categories, "text": text})


{'sections': [{'section': 'Information that I saw online about COVID-19 disease causing sterility in the summer was removed.',
   'category': 'bias',
   'sub_category': 'misinformation',
   'explanation': 'The statement suggests biased or misleading information about COVID-19 causing sterility.'},
  {'section': 'I heard it’s because of lawsuits related to Bill Gates and the vaccines in Africa.',
   'category': 'bias',
   'sub_category': 'conspiracy theory',
   'explanation': 'The mention of Bill Gates and vaccines in Africa hints at a biased conspiracy theory.'},
  {'section': 'Does this have to do with the COVID-19 vaccines causing infertility?',
   'category': 'bias',
   'sub_category': 'misinformation',
   'explanation': 'The question implies biased information about COVID-19 vaccines causing infertility without factual basis.'}]}